In [ ]:
#baseline model UNET
# I splitted every image to patches of 64 * 64 pixels 
#and classified every pixel in every patch as binary classification  task


#so for the model the input is 64 * 64 pixels images.


#model: UNET
#loss Binary cross entropy 
#optimizer: SGD

#I splitted to dataset under to dicom-images-train

In [18]:
from torch.utils.data import Dataset
from skimage import io, transform
import os
import pandas as pd
from PIL import Image
import random
import numpy as np
from torchvision import transforms, datasets
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from imageio import imread
import torch
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys
from mask_functions import rle2mask
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import pydicom

In [2]:

print('__pyTorch VERSION:', torch.__version__)
use_cuda = torch.cuda.is_available()
print('use_cuda = {0}'.format(use_cuda))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

DATA_FOLDER ='data'
device

__pyTorch VERSION: 1.10.1+cu102
use_cuda = True


'cuda'

In [3]:

SEED = 999

def fixSeed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

fixSeed(SEED)
SEED

999

In [10]:
df = pd.read_csv('archive/train-rle.csv', header=None, index_col=0)
train_fns = sorted(glob.glob('archive/dicom-images-train/*/*/*.dcm'))[0:500]
df_full = pd.read_csv('archive/train-rle.csv', index_col='ImageId')

In [11]:
im_height = 1024
im_width = 1024
im_chan = 1
X_s = np.zeros((len(train_fns), im_height, im_width, im_chan), dtype=np.uint8)
Y_s = np.zeros((len(train_fns), im_height, im_width, 1), dtype=np.bool)
sys.stdout.flush()
for n, _id in tqdm(enumerate(train_fns), total=len(train_fns)):
    dataset = pydicom.read_file(_id)
    X_s[n] = np.expand_dims(dataset.pixel_array, axis=2)
    try:
        if '-1' in df_full.loc[_id.split('/')[-1][:-4],' EncodedPixels']:
            Y_s[n] = np.zeros((1024, 1024, 1))
        else:
            if type(df_full.loc[_id.split('/')[-1][:-4],' EncodedPixels']) == str:
                Y_s[n] = np.expand_dims(rle2mask(df_full.loc[_id.split('/')[-1][:-4],' EncodedPixels'], 1024, 1024), axis=2)
            else:
                Y_s[n] = np.zeros((1024, 1024, 1))
                for x in df_full.loc[_id.split('/')[-1][:-4],' EncodedPixels']:
                    Y_s[n] =  Y_s[n] + np.expand_dims(rle2mask(x, 1024, 1024), axis=2)
    except KeyError:
        print(f"Key {_id.split('/')[-1][:-4]} without mask, assuming healthy patient.")
        Y_s[n] = np.zeros((1024, 1024, 1)) # Assume missing masks are empty masks.
        

 55%|█████▍    | 273/500 [00:02<00:01, 139.56it/s]

Key 1.2.276.0.7230010.3.1.4.8323329.10231.1517875222.737143 without mask, assuming healthy patient.


 85%|████████▍ | 424/500 [00:03<00:00, 144.41it/s]

Key 1.2.276.0.7230010.3.1.4.8323329.10362.1517875223.377845 without mask, assuming healthy patient.


 94%|█████████▍| 470/500 [00:03<00:00, 142.54it/s]

Key 1.2.276.0.7230010.3.1.4.8323329.10407.1517875223.567351 without mask, assuming healthy patient.


100%|██████████| 500/500 [00:03<00:00, 137.41it/s]


In [12]:
X_s.shape, Y_s.shape

((500, 1024, 1024, 1), (500, 1024, 1024, 1))

In [13]:
validation_indexes

array([], dtype=float64)

In [14]:
validation_size = int(Y_s.shape[0] * 0.1)
validation_indexes = np.random.choice(range(0, Y_s.shape[0]), size=validation_size, replace=False)
train_indexes  = [i for i in range(0, Y_s.shape[0]) if i not in validation_indexes]


y_train = Y_s[train_indexes, :, :]
y_validation = Y_s[validation_indexes, :, :]



X_validation = X_s[validation_indexes, :, :]
X_train = X_s[train_indexes, :, :]

y_train.shape, X_train.shape

((450, 1024, 1024, 1), (450, 1024, 1024, 1))

In [15]:
im_height = 64
im_width = 64
X_train = X_train.reshape((-1, im_height, im_width, 1))
y_train = y_train.reshape((-1, im_height, im_width, 1))


X_validation = X_validation.reshape((-1, im_height, im_width, 1))
y_validation = y_validation.reshape((-1, im_height, im_width, 1))

X_train.shape, y_train.shape

((115200, 64, 64, 1), (115200, 64, 64, 1))

In [16]:
class DataGenerator(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks
        

    def __len__(self):
        return self.masks.shape[0]

    def __getitem__(self, index):
        
        y = self.masks[index]
        y = np.where(y == True, 1, 0)
        
        image  = X_train[index]
        image = image.reshape(1, image.shape[0], image.shape[1])
        y = y.reshape(1, y.shape[0], y.shape[1])
        
        return image, y

In [17]:
training_generator = DataGenerator(X_train, y_train)
validation_generator = DataGenerator(X_validation, y_validation)

In [26]:
y_train.shape, X_train.shape

((115200, 64, 64, 1), (115200, 64, 64, 1))

In [21]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


In [26]:
def train(model, data_loader, confusion_matrix, optimizer, criterion, input_transformation=None):
    calc_loss = 0.
    calc_count = 0.

    model.train()

    for images, labels in data_loader:

        calc_count += labels.shape[0]
        images = images.float().to(device)
     
        labels = labels.float().to(device)
        images = Variable(images)
        labels = Variable(labels)

        output = model(images)
        loss = criterion(output, labels)
        calc_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return calc_loss/calc_count


def eval_model(model, data_loader, criterion):
    model.eval()
    
    calc_loss = 0.
    calc_count = 0.
    
    with torch.no_grad():
        for images, labels in data_loader:
            calc_count += labels.shape[0]
            images = images.float().to(device)

            labels = labels.float().to(device)
            images = Variable(images)
            labels = Variable(labels)
            output = model(images)
            loss = criterion(output, labels)
            calc_loss += loss.item()
            
        
    return calc_loss/calc_count

    



In [29]:
model = UNet(1, 1, False).to(device)
lr = 0.001
batch_size = 16
num_epochs = 50
momentum = 0.9
number_of_epoch = 100 



In [31]:
criterion =nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)


data_loader = DataLoader(training_generator, batch_size=8, shuffle=True, num_workers=0)
validation_data_loader = DataLoader(validation_generator, batch_size=8, shuffle=True, num_workers=0)


for epoch in range(10):
    print(epoch)
    train_loss = train(model, data_loader, None, optimizer, criterion, None)
    val_loss = eval_model(model, validation_data_loader, criterion)
    
    print('epoch {} train loss {} val loss {}'.format(epoch, train_loss, val_loss))